In [3]:
!pip install requests
!pip install BeautifulSoup4
!pip install wikipedia
print('Installed')

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
Installed


In [85]:
import pandas as pd
import wikipedia as wp
import numpy as np
import requests

from bs4 import BeautifulSoup

html = wp.page("List_of_postal_codes_of_Canada:_M").html()
df = pd.read_html(html)[0]
print(df.to_string())

    Postcode           Borough                                      Neighbourhood
0        M1A      Not assigned                                       Not assigned
1        M2A      Not assigned                                       Not assigned
2        M3A        North York                                          Parkwoods
3        M4A        North York                                   Victoria Village
4        M5A  Downtown Toronto                                       Harbourfront
5        M6A        North York                                   Lawrence Heights
6        M6A        North York                                     Lawrence Manor
7        M7A  Downtown Toronto                                       Queen's Park
8        M8A      Not assigned                                       Not assigned
9        M9A         Etobicoke                                   Islington Avenue
10       M1B       Scarborough                                              Rouge
11       M1B    

In [83]:
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
Postcode         287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 6.8+ KB


In [90]:
test = df
test[test.Borough.str.contains("Not assigned") == False]



,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North
